In [1]:
# Import SparkSession
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, to_timestamp, regexp_replace
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [2]:
# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [3]:
def merge_all(dfs):
    
    if len(dfs) == 0:
        return None
    
    df = dfs[0]
    
    for df2 in dfs[1:]:
        df = df.union(df2)
    
    return df

# Atracação

In [4]:
df_atracacao_2020 = spark.read.options(header='True', inferSchema='True', delimiter=';') \
  .csv("data/raw/2020Atracacao.txt")

df_atracacao_2019 = spark.read.options(header='True', inferSchema='True', delimiter=';') \
  .csv("data/raw/2019Atracacao.txt")

df_atracacao_2018 = spark.read.options(header='True', inferSchema='True', delimiter=';') \
  .csv("data/raw/2018Atracacao.txt")

dfs = [df_atracacao_2020, df_atracacao_2019, df_atracacao_2018]

In [5]:
df_atracacao = merge_all(dfs)

In [6]:
df_atracacao.dtypes

[('IDAtracacao', 'int'),
 ('CDTUP', 'string'),
 ('IDBerco', 'string'),
 ('Berço', 'string'),
 ('Porto Atracação', 'string'),
 ('Apelido Instalação Portuária', 'string'),
 ('Complexo Portuário', 'string'),
 ('Tipo da Autoridade Portuária', 'string'),
 ('Data Atracação', 'string'),
 ('Data Chegada', 'string'),
 ('Data Desatracação', 'string'),
 ('Data Início Operação', 'string'),
 ('Data Término Operação', 'string'),
 ('Ano', 'int'),
 ('Mes', 'string'),
 ('Tipo de Operação', 'string'),
 ('Tipo de Navegação da Atracação', 'string'),
 ('Nacionalidade do Armador', 'int'),
 ('FlagMCOperacaoAtracacao', 'int'),
 ('Terminal', 'string'),
 ('Município', 'string'),
 ('UF', 'string'),
 ('SGUF', 'string'),
 ('Região Geográfica', 'string'),
 ('Nº da Capitania', 'string'),
 ('Nº do IMO', 'int')]

In [7]:
# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names):
    for name in names: 
        df = df.withColumn(name, to_timestamp(df[name], 'dd/MM/yyyy HH:mm:ss'))
    return df 

# Assign all column names to `columns`
columns = ['Data Atracação', 'Data Chegada','Data Desatracação', 'Data Início Operação', 'Data Término Operação']

# Conver the `df` columns to `FloatType()`
df_atracacao = convertColumn(df_atracacao, columns)
                           
# print(df_atracacao.dtypes)

# df_atracacao.select('Data Chegada').show(20)

## TEsperaAtracacao: Atracação - Chegada

In [8]:
df_atracacao = df_atracacao.withColumn(
    "TEsperaAtracacao", 
    (F.col("Data Atracação").cast("long") - F.col("Data Chegada").cast("long"))/60.
)

# cols = ['Data Atracação','Data Chegada','TEsperaAtracacao']
# df_atracacao.select(*cols).show(25)


## TEsperaInicioOp:  Início - Atracação

In [9]:
df_atracacao = df_atracacao.withColumn(
    "TEsperaInicioOp", 
    (F.col("Data Início Operação").cast("long") - F.col("Data Atracação").cast("long"))/60.
)

# cols = ['Data Início Operação','Data Atracação','TEsperaInicioOp']
# df_atracacao.select(*cols).show(25)


## TOperacao: Término - Início

In [10]:
df_atracacao = df_atracacao.withColumn(
    "TOperacao", 
    (F.col("Data Término Operação").cast("long") - F.col("Data Início Operação").cast("long"))/60.
)

# cols = ['Data Término Operação','Data Início Operação','TOperacao']
# df_atracacao.select(*cols).show(25)


## TEsperaDesatracacao: Desatracação - Término

In [11]:
df_atracacao = df_atracacao.withColumn(
    "TEsperaDesatracacao", 
    (F.col("Data Desatracação").cast("long") - F.col("Data Término Operação").cast("long"))/60.
)

# cols = ['Data Desatracação','Data Término Operação','TEsperaDesatracacao']
# df_atracacao.select(*cols).show(25)


## TAtracado: Desatracação - Atracação

In [12]:
df_atracacao = df_atracacao.withColumn(
    "TAtracado", 
    (F.col("Data Desatracação").cast("long") - F.col("Data Atracação").cast("long"))/60.
)

# cols = ['Data Desatracação','Data Atracação','TAtracado']
# df_atracacao.select(*cols).show(25)


## TEstadia: Desatracação - Chegada

In [13]:
df_atracacao = df_atracacao.withColumn(
    "TEstadia", 
    (F.col("Data Desatracação").cast("long") - F.col("Data Chegada").cast("long"))/60.
)

# cols = ['Data Desatracação','Data Chegada','TEstadia']
# df_atracacao.select(*cols).show(25)


In [15]:
df_atracacao.write.option("quoteAll", True).csv("data/processed/atracacao_fato.csv", header=True)

# Carga

In [16]:
df_carga_2020 = spark.read.options(header='True', inferSchema='True', delimiter=';') \
  .csv("data/raw/2020Carga.txt")

df_carga_2019 = spark.read.options(header='True', inferSchema='True', delimiter=';') \
  .csv("data/raw/2019Carga.txt")

df_carga_2018 = spark.read.options(header='True', inferSchema='True', delimiter=';') \
  .csv("data/raw/2018Carga.txt")

dfs = [df_carga_2020, df_carga_2019, df_carga_2018]

In [17]:
df_carga = merge_all(dfs)

In [18]:

df_carga = df_carga.join(df_atracacao, df_carga.IDAtracacao == df_atracacao.IDAtracacao).select(df_carga["*"], df_atracacao["Ano"], df_atracacao["Mes"], df_atracacao["Porto Atracação"], df_atracacao["SGUF"])


### Peso líquido da carga (Carga não conteinerizada = Peso bruto e Carga conteinerizada = Peso sem contêiner)

In [19]:
commaToDot = udf(lambda x : float(str(x).replace(',', '.')), FloatType())

df_carga = df_carga.withColumn(
    "VLPesoCargaBruta", 
    commaToDot('VLPesoCargaBruta')
)

In [20]:
df_carga = df_carga.withColumn(
    "Peso líquido da carga", 
    col("VLPesoCargaBruta")
)


df_carga = df_carga.withColumn(
    "Peso líquido da carga", 
    F.when( (col("FlagConteinerTamanho") == '20')  & (col("FlagConteinerTamanho").isNotNull()),
           (col("VLPesoCargaBruta") - 2.3)).otherwise(F.when( (col("FlagConteinerTamanho") == '40'),
           (col("VLPesoCargaBruta") - 3.7)).otherwise(col("VLPesoCargaBruta")))
)


In [21]:
df_carga.write.option("quoteAll", True).csv("data/processed/carga_fato.csv", header=True)